In [125]:
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from imblearn.combine import SMOTETomek
import MachineLearning as ML
import PrepareData
import DB_reader

import importlib

In [126]:
importlib.reload(PrepareData)

df=PrepareData.get_data()

df=df.dropna()

C:\Users\Adam\source\repos\WeatherPrediction\WeatherPredictionProject\PrepareData.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = df['datetime'].dt.month
C:\Users\Adam\source\repos\WeatherPrediction\WeatherPredictionProject\PrepareData.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df = df.drop(columns=['datetime'], axis=1)
C:\Users\Adam\source\repos\WeatherPrediction\WeatherPredictionProject\PrepareData.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [127]:
importlib.reload(ML)

def get_models(X_train, y_train):
    models=[]

    X_train, y_train = SMOTETomek().fit_resample(X_train, y_train)

    models.append(ML.NeuralNetworks(X_train, y_train))
    models.append(ML.KNN(X_train, y_train))
    models.append(ML.DecisionTree(X_train, y_train))
    models.append(ML.LinearRegressionModel(X_train, y_train))
    models.append(ML.LogisticRegressionModel(X_train, y_train))
    models.append(ML.RandomForest(X_train, y_train))
    models.append(ML.SupportVectorMachines(X_train, y_train))
    models.append(ML.SupportVectorRegression(X_train, y_train))
    models.append(ML.TreeGradientBoosting(X_train, y_train))

    return models

In [128]:
y=df['nextday_rainfall']
X=df.drop('nextday_rainfall', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.25, stratify=y)

In [129]:
models=get_models(X_train, y_train)
results=[]

today_data=DB_reader.getWeatherForecast()
today_data=PrepareData.standarize_row(today_data, df.iloc[:,1:].columns.values)


for model in models:
    results.append(model.predict(today_data)>0.5)


# cm = confusion_matrix(y_test, y_pred)
# print(cm)
#
# f1_sc = f1_score(y_test.values.ravel(), y_pred.ravel())
# print(f1_sc)

Epoch 1/10
656/656 [==============================] - 3s 3ms/step - loss: 0.6940 - f1: 0.3416 - accuracy: 0.5034 - val_loss: 0.6936 - val_f1: 0.6445 - val_accuracy: 0.5000
Epoch 2/10
656/656 [==============================] - 2s 3ms/step - loss: 0.6938 - f1: 0.3966 - accuracy: 0.4897 - val_loss: 0.6935 - val_f1: 0.0000e+00 - val_accuracy: 0.5000
Epoch 3/10
656/656 [==============================] - 2s 3ms/step - loss: 0.6935 - f1: 0.3141 - accuracy: 0.5065 - val_loss: 0.6938 - val_f1: 0.0000e+00 - val_accuracy: 0.5000
Epoch 4/10
656/656 [==============================] - 2s 2ms/step - loss: 0.6925 - f1: 0.3184 - accuracy: 0.5046 - val_loss: 0.6828 - val_f1: 0.6999 - val_accuracy: 0.6749
Epoch 5/10
656/656 [==============================] - 2s 2ms/step - loss: 0.6722 - f1: 0.6138 - accuracy: 0.5846 - val_loss: 0.6232 - val_f1: 0.6732 - val_accuracy: 0.6949
Epoch 6/10
656/656 [==============================] - 2s 2ms/step - loss: 0.6570 - f1: 0.6595 - accuracy: 0.6294 - val_loss: 0.6223 

C:\Users\Adam\source\repos\WeatherPrediction\WeatherPredictionProject\PrepareData.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = df['datetime'].dt.month
C:\Users\Adam\source\repos\WeatherPrediction\WeatherPredictionProject\PrepareData.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df = df.drop(columns=['datetime'], axis=1)
C:\Users\Adam\source\repos\WeatherPrediction\WeatherPredictionProject\PrepareData.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

1/1 [==============================] - 0s 132ms/step


In [130]:
print("Jutro w Olsztynie będzie padał deszcz lub śnieg") if (sum(results)/len(results))>0.5 else print("Jutro w Olsztynie nie będzie opadów deszczu")

Jutro w Olsztynie będzie padał deszcz lub śnieg
